In [ ]:

import os
import librosa
import soundfile as sf

input_dir = 'data/raw_audio'
output_dir = 'data/processed_audio'
os.makedirs(output_dir, exist_ok=True)

def preprocess_audio(file_path, output_path):
    y, sr = librosa.load(file_path, sr=None)
    y_trimmed, _ = librosa.effects.trim(y)
    y_denoised = librosa.effects.preemphasis(y_trimmed)
    sf.write(output_path, y_denoised, sr)

for file_name in os.listdir(input_dir):
    if file_name.endswith('.wav'):
        input_path = os.path.join(input_dir, file_name)
        output_path = os.path.join(output_dir, file_name)
        preprocess_audio(input_path, output_path)


In [ ]:

import os
import librosa
import numpy as np
import pandas as pd

input_dir = 'data/processed_audio'
features = []

for file_name in os.listdir(input_dir):
    if file_name.endswith('.wav'):
        file_path = os.path.join(input_dir, file_name)
        y, sr = librosa.load(file_path, sr=None)
        mfcc = np.mean(librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13).T, axis=0)
        zcr = np.mean(librosa.feature.zero_crossing_rate(y).T, axis=0)
        rms = np.mean(librosa.feature.rms(y=y).T, axis=0)
        stft = np.mean(librosa.feature.spectral_centroid(y=y, sr=sr).T, axis=0)
        feature_vector = np.hstack((mfcc, zcr, rms, stft))
        label = file_name.split('_')[0]  # assuming label prefix
        features.append([label] + list(feature_vector))

df = pd.DataFrame(features)
df.to_csv('features/extracted_features.csv', index=False)


In [ ]:

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.utils import to_categorical

# Load features
df = pd.read_csv('features/extracted_features.csv')
X = df.iloc[:, 1:]
y = df.iloc[:, 0]

# Encode labels
encoder = LabelEncoder()
y_encoded = encoder.fit_transform(y)
y_cat = to_categorical(y_encoded)

# Scale features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_cat, test_size=0.2, stratify=y_cat)

# Model
model = Sequential([
    Dense(256, activation='relu', input_shape=(X.shape[1],)),
    Dropout(0.3),
    Dense(128, activation='relu'),
    Dropout(0.3),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(32, activation='relu'),
    Dense(y_cat.shape[1], activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=100, validation_split=0.2, batch_size=32)
model.save('models/dnn_model.h5')


In [ ]:

import pandas as pd
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.models import load_model
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

# Load features
df = pd.read_csv('features/extracted_features.csv')
X = df.iloc[:, 1:]
y = df.iloc[:, 0]

# Encode labels
encoder = LabelEncoder()
y_encoded = encoder.fit_transform(y)
y_cat = to_categorical(y_encoded)

# Scale features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_cat, test_size=0.2, stratify=y_cat)

# Load model
model = load_model('models/dnn_model.h5')
y_pred = model.predict(X_test)
y_pred_classes = y_pred.argmax(axis=1)
y_true_classes = y_test.argmax(axis=1)

# Report
print(classification_report(y_true_classes, y_pred_classes))
